In [85]:
import spacy
import pandas as pd

In [86]:
with open("20th_century_events.txt", "r") as file:
    text = file.read()

In [87]:
text[:500]

"The 20th century changed the world in unprecedented ways. The World Wars sparked tension between countries and led to the creation of atomic bombs , the Cold War led to the Space Race and the creation of space-based rockets, and the World Wide Web was created. These advancements have played a significant role in citizens' lives and shaped the 21st century into what it is today. Historic events in the 20th century [ edit ] World at the beginning of the century [ edit ] Main article: Edwardian era"

In [88]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

In [89]:
[(ent.text, ent.label_) for ent in doc.ents[:20]]

[('The 20th century', 'DATE'),
 ('the Cold War', 'EVENT'),
 ('the Space Race', 'WORK_OF_ART'),
 ('the World Wide Web', 'WORK_OF_ART'),
 ('the 21st century', 'DATE'),
 ('today', 'DATE'),
 ('the 20th century', 'DATE'),
 ('the beginning of the century', 'DATE'),
 ('the 20th century', 'DATE'),
 ('The 1900s', 'DATE'),
 ('the decade', 'DATE'),
 ('the Panama Canal', 'ORG'),
 ('Scramble', 'GPE'),
 ('Africa', 'LOC'),
 ('the 1900s', 'DATE'),
 ('the Congo Free State', 'FAC'),
 ('1914 to 1918', 'DATE'),
 ('the First World War', 'EVENT'),
 ('World War I', 'EVENT'),
 ('World War I Arrest', 'EVENT')]

In [90]:
sentences = list(doc.sents)
sentences[:3]   

[The 20th century changed the world in unprecedented ways.,
 The World Wars sparked tension between countries and led to the creation of atomic bombs , the Cold War led to the Space Race and the creation of space-based rockets, and the World Wide Web was created.,
 These advancements have played a significant role in citizens' lives and shaped the 21st century into what it is today.]

In [2]:
import spacy
import pandas as pd

# Load the text file
with open("20th_century_events.txt", "r") as file:
    text = file.read()

# Create NER object
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Split into sentences
sentences = list(doc.sents)

# Extract countries LOC and GPE
countries = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]]
countries = list(set(countries))

sentence_countries = []

for sent in sentences:
    countries_in_sentence = [
        ent.text for ent in sent.ents 
        if ent.label_ in ["GPE", "LOC"]
    ]
    if len(countries_in_sentence) > 1:
        sentence_countries.append(countries_in_sentence)



relationship_pairs = []

for country_list in sentence_countries:
    for i in range(len(country_list)):
        for j in range(i + 1, len(country_list)):
            relationship_pairs.append({
                "source": country_list[i],
                "target": country_list[j],
                "value": 1
            })

relationships_df = pd.DataFrame(relationship_pairs)

relationships_df = relationships_df.groupby(
    ["source", "target"], 
    as_index=False
).sum()

relationships_df

relationships_df

,source,target,value
0,Africa,Algeria,1
1,Africa,Asia,2
2,Africa,Ghana,1
3,Africa,Kenya,1
4,Africa,Morocco,1
...,...,...,...
424,the United States,North Korea,1
425,the United States,Pakistan,1
426,the United States,Russia,2
427,the United States,the Soviet Union,3


The text contains a few issues from the scrape, such as [ edit ] and long dashes. Some extracted items are not actual countries (e.g., Scramble, the Central Powers), and some country names include extra words like “the.” These inconsistencies need light cleaning to ensure accurate analysis.

In [92]:
clean_text = text.replace("[ edit ]", "")
clean_text = clean_text.replace("–", "-")

In [93]:
with open("20th_century_events_clean.txt", "w") as file:
    file.write(clean_text)

In [94]:
import spacy


with open("20th_century_events_clean.txt", "r") as file:
    text = file.read()

# Create  NER object
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Show a few entities
[(ent.text, ent.label_) for ent in doc.ents[:15]]

[('The 20th century', 'DATE'),
 ('the Cold War', 'EVENT'),
 ('the Space Race', 'WORK_OF_ART'),
 ('the World Wide Web', 'WORK_OF_ART'),
 ('the 21st century', 'DATE'),
 ('today', 'DATE'),
 ('the 20th century', 'DATE'),
 ('the beginning of the century', 'DATE'),
 ('Main', 'PRODUCT'),
 ('the 20th century', 'DATE'),
 ('The 1900s', 'DATE'),
 ('the decade', 'DATE'),
 ('the Panama Canal', 'ORG'),
 ('Scramble', 'GPE'),
 ('Africa', 'LOC')]

In [95]:
sentences = list(doc.sents)
sentences[:5]

[The 20th century changed the world in unprecedented ways.,
 The World Wars sparked tension between countries and led to the creation of atomic bombs , the Cold War led to the Space Race and the creation of space-based rockets, and the World Wide Web was created.,
 These advancements have played a significant role in citizens' lives and shaped the 21st century into what it is today.,
 Historic events in the 20th century  World at the beginning of the century  Main article: Edwardian era The new beginning of the 20th century marked significant changes.,
 The 1900s saw the decade herald a series of inventions, including the automobile , airplane and radio broadcasting .]

In [96]:
filtered_entities = [ent for ent in doc.ents if ent.text in countries]
filtered_entities[:20]

[Scramble,
 Africa,
 Sarajevo,
 Sarajevo,
 the British Empire,
 France,
 the Russian Empire,
 the Central Powers,
 German Empire,
 Austria,
 Russia,
 the Central Powers,
 Tsar,
 Bolsheviks,
 Germany,
 Russia,
 Germany,
 the Ottoman Empire,
 Ukraine,
 Germany]

In [97]:
import pandas as pd

df_relationships = pd.DataFrame({"Countries_in_Sentence": relationships})
df_relationships.head()

,Countries_in_Sentence
0,"[Scramble, Africa]"
1,"[the British Empire, France, the Russian Empir..."
2,"[Russia, the Central Powers, Tsar]"
3,"[Bolsheviks, Germany, Russia]"
4,"[Germany, the Ottoman Empire]"


In [98]:
df_relationships.to_csv("country_relationships.csv", index=False)